In [1]:
!pip install polars
!pip install seaborn
!pip install missingno
!pip install lightgbm
!pip install pyarrow

import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import missingno as mn
import datetime
import pyarrow

from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset


import matplotlib.pyplot as plt
from IPython import display

import os
import gc
from glob import glob
from pathlib import Path
import joblib

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")
TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

# Utils

In [3]:
def inspect_columns(df):
    df = df.to_pandas()

    result = pd.DataFrame({
        'unique': df.nunique() == len(df),
        'cardinality': df.nunique(),
        'with_null': df.isna().any(),
        'null_pct': round((df.isnull().sum() / len(df)) * 100, 2),
        '1st_row': df.iloc[0],
        'random_row': df.iloc[np.random.randint(low=0, high=len(df))],
        'last_row': df.iloc[-1],
        'dtype': df.dtypes
    })
    
    return result

def custom_format(x):
    if x % 1 == 0:
        return int(x)
    else:
        return '%.2f' % x

# Set display options
pd.set_option('display.float_format', custom_format)

In [4]:
def plot_distribution(
    column,

    dataframe,
    bins='auto',
    histplot = True,
    kdeplot = False,
    scatterplot = False,
    scatter_plot_y = False,

    xlim = False,
    ylim = False,
    
    is_categorical = False,
    countplot = True,
    pieplot = False,
    groupby = False,
    numeric_column_to_compare = False,
    categorical_column_to_compare = False,

    title=None,
    xlabel=None,
    ylabel=None,


    show_mean = False,
    show_median = False,
    show_mode = False,

    total_num = True,
    total_nan = True,
    count_zeros = True,
    total_non_nan_values = True,
    percentage_zeros = True,
    drop_invalid=True,
    describe=True
):
    
    """
    Plot the distribution of a column in a DataFrame.

    Parameters:
    - dataframe: pandas.DataFrame
    - column: str, the column name to plot
    - bins: int or 'auto', the number of bins for continuous data or 'auto' for automatic binning
    """

    # Check if the column exists in the DataFrame
    if column not in dataframe.columns:
        print(f"Error: Column '{column}' not found in the DataFrame.")
        return

    # Drop NaNs or empties if specified
    if drop_invalid:
        dataframe = dataframe.dropna(subset=[column])


    plt.figure(figsize=(10, 6))

    if is_categorical:
        if countplot:
            sns.countplot(x=dataframe[column], data=dataframe)
            plt.title(f'Distribution of {column} - Categorical')
            plt.ylabel('Frequency')
            plt.xlabel(column)
            plt.show()
        if pieplot:
            dataframe[column].value_counts().plot.pie(autopct='%1.1f%%')
            plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
            plt.show()

        if groupby:
            if numeric_column_to_compare:
                dataframe.groupby(column)[numeric_column_to_compare].mean().plot(kind='bar')
                plt.show()
            elif categorical_column_to_compare:
                # Your existing code
                sns.countplot(x=categorical_column_to_compare, hue=column, data=dataframe)
                # Adding x and y labels
                plt.xlabel('Credit Account Actual Balance')  # Replace with the actual label for the x-axis
                plt.ylabel('Count')  # Replace with the actual label for the y-axis
                plt.show()

    else:
        mean = dataframe[column].mean()
        median = dataframe[column].median()
        mode_value = statistics.mode(dataframe[column])

        if histplot:
            sns.histplot(dataframe[column], bins=bins, kde=kdeplot)
            plt.title(title or f'Distribution of {column}')
            plt.xlabel(xlabel or f'{column} values')
            plt.ylabel(ylabel or 'Frequency')
            plt.ylim(ylim)
            plt.xlim(xlim)

            if show_mean: plt.axvline(mean, color='red', linestyle='dashed', label='Mean')
            if show_median: plt.axvline(median, color='green', linestyle='dashed', label='Median')
            if show_mode: plt.axvline(mode_value, color='blue', linestyle='dashed', label='Mode')
            plt.legend()
            plt.show()
            
        if kdeplot:
            sns.kdeplot(dataframe[column])
            plt.xlabel(xlabel or f'{column} values')
            plt.ylabel(ylabel or 'Frequency')
            plt.ylim(ylim)
            plt.xlim(xlim)
            if show_mean: plt.axvline(mean, color='red', linestyle='dashed', label='Mean')
            if show_median: plt.axvline(median, color='green', linestyle='dashed', label='Median')
            if show_mode: plt.axvline(mode_value, color='blue', linestyle='dashed', label='Mode')
            plt.legend()
            plt.show()

        if scatterplot:
            sns.set_palette("husl")
            sns.set_theme(style="whitegrid")
            scatter_plot = sns.scatterplot(x=column, y=scatter_plot_y,  data=dataframe.iloc[:1000], palette="viridis", alpha=0.7)

            scatter_plot.set(
                xlabel=xlabel or f'{column} values',
                ylabel=ylabel or 'Frequency',
                title=title or f'Distribution of {column}'
            )
            
            scatter_plot.legend()
            scatter_plot.grid(True, linestyle='--', alpha=0.7)
            sns.regplot(x=column, y=scatter_plot_y, data=dataframe.iloc[:1000], scatter=False, ax=scatter_plot)
            plt.show()

    nzeros = (dataframe[column] == 0).sum()
    ntotalnonnan= dataframe[column].notna().sum()
    if total_num and not describe: print('Total: ', dataframe[column].shape[0])
    if total_non_nan_values: print('Total non zero value: ', ntotalnonnan)
    if total_nan: print('Missed values num. ',  dataframe[column].isnull().sum())
    if count_zeros and not is_categorical: print('Count zeros: ', nzeros)
    if percentage_zeros: print("Zero %: ", np.round(nzeros / ntotalnonnan * 100, 2))
    if show_mean and not is_categorical and  not describe: print("Mean: ", dataframe[column].mean())
    if show_median and not is_categorical and not describe: print('Median: ', dataframe[column].median())
    if show_mode: print('Mode: ', dataframe[column].mode()[0])
    if describe: dataframe[column].describe()


In [5]:
def scan_data_and_parse_dates(parquet_file):
    return pl.scan_parquet(parquet_file).with_columns(
        pl.col(r'^.*D$').str.to_datetime(),
    )

In [6]:
def plot_numerical(df, group, max_samples = 1000000):
    X = df.select(pl.col(f'^.*{group}$')).collect()
    if len(X) > max_samples:
        X = X.sample(max_samples, seed = 0)
    n = X.shape[1]
    fig, axes = plt.subplots(n, 2, figsize = (16, n * 4), squeeze = False)
    for i, c in enumerate(X.iter_columns()):
        x = c.to_pandas()
        sns.histplot(x = x, bins = 50, kde = True, ax = axes[i, 0])
        sns.boxplot(y = x, ax = axes[i, 1])
    fig.tight_layout()

In [7]:
def plot_categorical(df, group, max_categories = 30, max_samples = 1000000):
    X = df.select(pl.col(f'^.*{group}$')).collect()
    if len(X) > max_samples:
        X = X.sample(max_samples, seed = 0)
    n = X.shape[1]
    fig, axes = plt.subplots(n, 2, figsize = (16, n * 4), squeeze = False)
    for i, c in enumerate(X.iter_columns()):
        x = c.to_pandas()
        s = x.value_counts()
        if len(s) > max_categories:
            other_count = s[max_categories:].sum()
            s = s[:max_categories]
            s['Other'] = other_count
        sns.barplot(x = s.index, y = s, ax = axes[i, 0])
        axes[i, 1].pie(s, labels = s.index)
    fig.tight_layout()

In [8]:
def plot_corr(df, group, max_samples = 1000000):
    X = df.select(pl.col(f'^.*{group}$')).collect()
    if len(X) > max_samples:
        X = X.sample(max_samples, seed = 0)
    corr = X.to_pandas(use_pyarrow_extension_array = True).corr()
    w = len(corr)
    h = int(w * 0.8)
    _, ax = plt.subplots(figsize = (w, h))
    sns.heatmap(corr, ax = ax, vmin = -1, vmax = 1, annot = True, cmap = 'coolwarm')

In [9]:
def plot_null(df, group, max_samples = 1000000):
    X = df.select(pl.col(f'^.*{group}$')).collect()
    if len(X) > max_samples:
        X = X.sample(max_samples, seed = 0)
    n = max(X.shape[1], 8)
    X = X.to_pandas(use_pyarrow_extension_array = True)
    fig, axes = plt.subplots(2, figsize = (n, int(1.6 * n)))
    mn.matrix(X, ax = axes[0], sparkline = False)
    mn.heatmap(X, ax = axes[1])
    fig.tight_layout()

In [10]:
def plot_numerical_by_target(df_base, df, group, max_samples = 1000000):
    df_base = df_base.lazy()
    X = df_base.join(
        df, on = 'case_id', how = 'left',
    ).select('target', pl.col(f'^.*{group}$')).collect()
    if len(X) > max_samples:
        X = X.sample(max_samples, seed = 0)
    y = X['target'].to_pandas()
    X = X.drop('target')
    n = X.shape[1]
    fig, axes = plt.subplots(n, 2, figsize = (16, n * 4), squeeze = False)
    for i, c in enumerate(X.iter_columns()):
        x = c.to_pandas()
        sns.violinplot(x = y, y = x, orient = 'v', order = [0, 1], ax = axes[i, 0])
        sns.stripplot(x = y, y = x, orient = 'v', order = [0, 1], ax = axes[i, 1],
                      marker = '.')
    fig.tight_layout()

In [11]:
def plot_categorical_by_target(df_base, df, group, max_samples=1000000):
    df_base = df_base.lazy()
    X = df_base.join(
            df, on = 'case_id', how = 'left',
        ).select('target', pl.col(f'^.*{group}$')).collect()
    
    if len(X) > max_samples:
        X = X.sample(max_samples, seed=0)

    y = X['target'].to_pandas()
    X = X.drop('target')

    n = X.shape[1]
    fig, axes = plt.subplots(n, 2, figsize=(16, n * 4), squeeze=False)

    for i, c in enumerate(X.iter_columns()):
        x = c.to_pandas()
        sns.violinplot(x=y, y=x, orient='v', order=[0, 1], ax=axes[i, 0])
        # sns.stripplot(x=y, y=x, orient='v', order=[0, 1], ax=axes[i, 1], marker='.')

    fig.tight_layout()


In [12]:
def check_ttest_by_target(ids, base):
    filtered_targets = base.filter(base['case_id'].is_in(ids))['target']
    base_targets = base['target']
    
    
    filtered_targets_percent = filtered_targets.sum() / len(filtered_targets)
    base_targets_percent = base_targets.sum() / len(base)
    
    t_statistic, p_value = scipy.stats.ttest_ind(filtered_targets, base_targets)
 
    print("T-statistic:", t_statistic)
    print("P-value:", p_value)
    print("Unique targets percent:", filtered_targets_percent)
    print("Base targets percents:", base_targets_percent)
    
    return t_statistic, p_value

# Prev aplication concatenation 

In [13]:
# df_train_applprev_1 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_1_*.parquet')
# df_train_applprev_2 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_2.parquet')

In [14]:
# app_prev1 = df_train_applprev_1.lazy().collect()
# app_prev2 = df_train_applprev_2.lazy().collect()

# import pickle
# encoded_decoded  = {
#      '0': 'null',
#      '1': 'PHONE',
#      '2': 'ALTERNATIVE_PHONE',
#      '3': 'PRIMARY_EMAIL',
#      '4': 'SECONDARY_MOBILE',
#      '5': 'EMPLOYMENT_PHONE',
#      '6': 'WHATSAPP',
#      '7': 'PRIMARY_MOBILE',
#      '8': 'SKYPE',
#      '9': 'HOME_PHONE'
# }

# decoded_encoded = {v: k for k, v in encoded_decoded.items()}
# phone_encoder_pickle_path = 'phone_decoder.pickle'
# with open(phone_encoder_pickle_path, 'wb') as file:
#     # Dump the dictionary into the file
#     pickle.dump(encoded_decoded, file)
    
# with open(phone_encoder_pickle_path, 'rb') as file:
#     # Load the dictionary from the file
#     loaded_dict = pickle.load(file)

In [15]:
# def extract_conts_type_509L(list_of_series):
#     array = []
#     for item in list_of_series[0]: 
#         array.append(decoded_encoded[item if item else "null"])
#     return ",".join(array)

In [16]:
# def extract_cacccardblochreas_147M(list_of_series):
#     for value in list_of_series[0]:
#         if value is not None:
#             return value
#     return None

In [17]:
# def extract_credacc_cards_status_52L(list_of_series):
#     tokens = []
#     for value in list_of_series[0]:
#         if value is not None:
#              tokens.append(value)
#     return ','.join(tokens) if len(tokens) > 0 else None

In [18]:
# result_df = app_prev2.group_by(['case_id', 'num_group1']).agg(
#     pl.map_groups(
#             exprs=["conts_type_509L"],
#             function=extract_conts_type_509L
#         ).alias("conts_type_509L"),
#     pl.map_groups(
#             exprs=["cacccardblochreas_147M"],
#             function=extract_cacccardblochreas_147M
#         ).alias("cacccardblochreas_147M"),
#     pl.map_groups(
#             exprs=["credacc_cards_status_52L"],
#             function=extract_credacc_cards_status_52L,
#         ).alias("credacc_cards_status_52L")
# )

In [19]:

# result_df = result_df.with_columns(
#     result_df['credacc_cards_status_52L'].map_elements(lambda value: value[0] if isinstance(value, list) else value, skip_nulls=False),
#     result_df['cacccardblochreas_147M'].map_elements(lambda value: value[0] if isinstance(value, list) else value, skip_nulls=False),
# )


In [20]:
# result_df['cacccardblochreas_147M'].unique()
# # result_df['credacc_cards_status_52L'].unique()/

In [21]:
# print(len(df_train_applprev_1))
# print(len(joined_df))
# print(len(result_df))

In [22]:
# joined_df = app_prev1.join( ## move up
#     result_df,
#     on=["case_id", "num_group1"],
#     how="inner"  # You can change the join type to "left", "right", "outer", or "cross" as needed
# )
# joined_df.write_parquet('app_prev.parquet') 
# joined_df

# Bureau A

In [23]:
#                       a_1    /   a_2

# Counts in depth 1: 15_940_537 /  7_580_244 - a1 has at twice as much recors as a_2 after collapsing
# Unique case_id: 1_386_273 / 1_385_288 - a_1 completely cover all a_2 unique id 985

In [24]:
# df_train_credit_bureau_a_1 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_credit_bureau_a_1_*.parquet')
# credit_bureau_a_1 = df_train_credit_bureau_a_1.lazy().collect()

In [25]:
# credit_bureau_a_2 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_credit_bureau_a_2_*.parquet').lazy().collect() 
# cread_var = scan_data_and_parse_dates('/kaggle/working/bereau_a_2(1)_1.parquet').lazy().collect() 

In [26]:
# def find_ceil_sum(list_of_series):
#     return np.ceil(np.nan_to_num(np.array(list_of_series[0]))).sum().astype('int')
    

In [27]:
# def find_average(list_of_series):
#     non_zero_values = np.trim_zeros(np.nan_to_num(np.array(list_of_series[0])))
#     length = len(non_zero_values)
#     return non_zero_values.sum() / length if length > 0 else 0.
    

In [28]:
# def pmts_overdue_active_day_percent(list_of_series):
#     list_array = np.ceil(np.nan_to_num(np.array(list_of_series[0]))).astype('int')
#     list_len = len(list_array)
    
#     unique, counts = np.unique(list_array, return_counts=True)
#     dictOfUniqueValue = dict(zip(unique, counts))
#     is_null_in_dict = 0 in dictOfUniqueValue
 
#     return (1 - dictOfUniqueValue[0] / list_len) if is_null_in_dict else 1.
    

In [29]:
# def pmts_overdue_weighted_average(list_of_series):
#     list_array = np.ceil(np.nan_to_num(np.array(list_of_series[0])))
#     list_len = len(list_array)
    
#     return list_array.sum() / list_len

In [30]:
# def getMode(list_of_series):
#     mode = stats.mode(np.array(list_of_series[0])).mode
#     return int(mode) if not np.isnan(mode) else 0

In [31]:
# def getStartDate(list_of_series):
#     years = np.nan_to_num(np.array(list_of_series[0])).astype('int')
#     if years.sum() == 0: 
#         return 0
    

#     year = years[years != 0].min()
    
#     months = np.nan_to_num(np.array(list_of_series[1])).astype('int')
#     month = np.trim_zeros(months)[0]
    
#     if month:
#         return datetime.datetime(year, month, 1)
#     else:
#         return datetime.datetime(year, 1, 1)

In [32]:
# def getEndDate(list_of_series):
#     years = np.nan_to_num(np.array(list_of_series[0])).astype('int')
#     if years.sum() == 0: 
#         return 0
    
#     year = years.max()
    
#     #convert fill nan with zeros and convert it from float to int
#     months = np.nan_to_num(np.array(list_of_series[1])).astype('int')

#     # get first non-zero value from the end
#     month = np.trim_zeros(months[::-1])[0]
    
#     if month:
#         return datetime.datetime(year, month, 1)
#     else:
#         return datetime.datetime(year, 1, 1)

In [33]:
# def select_mask(list_of_series):
#     rawArray = np.array(list_of_series)
#     filtered_arr = rawArray[rawArray != "a55475b1"] 
#     return filtered_arr[0] if len(filtered_arr) > 0 else 'a55475b1'

In [34]:
# already_done = 6
# for i in range(11 - already_done):
#     # Read and parse data
#     credit_bureau_a_2 = scan_data_and_parse_dates(f'/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_credit_bureau_a_2_{i+already_done}.parquet').lazy().collect() 

#     # Group by 'case_id' and 'num_group1' and perform aggregations
#     result_df = credit_bureau_a_2.group_by(['case_id', 'num_group1']).agg(
#         # Active
#         pl.map_groups(exprs=["pmts_year_1139T"], function=getMode).alias("pmts_year_active_mode"),
#         pl.map_groups(exprs=["pmts_year_1139T", "pmts_month_158T"], function=getStartDate).alias("pmts_year_active_start"), 
#         pl.map_groups(exprs=["pmts_year_1139T", "pmts_month_158T"], function=getEndDate).alias("pmts_year_active_finish"), 

#         pl.map_groups(exprs=["pmts_overdue_1140A"], function=find_ceil_sum).alias("pmts_overdue_active_sum"),
#         pl.map_groups(exprs=["pmts_overdue_1140A"], function=pmts_overdue_active_day_percent).alias("pmts_overdue_active_day_percent"),
#         pl.map_groups(exprs=["pmts_overdue_1140A"], function=pmts_overdue_weighted_average).alias("pmts_overdue_active_weighted_average"),

#         pl.map_groups(exprs=["pmts_dpd_1073P"], function=find_ceil_sum).alias("pmts_dpd_active"),

#         pl.map_groups(exprs=["collater_valueofguarantee_1124L"], function=find_average).alias("collater_value_active"),

#         pl.map_groups(exprs=["collater_typofvalofguarant_298M"], function=select_mask).alias("collater_typofvalofguarant_active"),

#         pl.map_groups(exprs=["collaterals_typeofguarante_669M"], function=select_mask).alias("collaterals_typeofguarante_active"),

#         pl.map_groups(exprs=["subjectroles_name_838M"], function=select_mask).alias("subjectroles_name_active"),

#         # Closed
#         pl.map_groups(exprs=["pmts_year_507T"], function=getMode).alias("pmts_year_closed_mode"),
#         pl.map_groups(exprs=["pmts_year_507T", "pmts_month_706T"], function=getStartDate).alias("pmts_year_closed_start"), 
#         pl.map_groups(exprs=["pmts_year_507T", "pmts_month_706T"], function=getEndDate).alias("pmts_year_closed_finish"), 

#         pl.map_groups(exprs=["pmts_overdue_1152A"], function=find_ceil_sum).alias("pmts_overdue_closed_sum"),
#         pl.map_groups(exprs=["pmts_overdue_1152A"], function=pmts_overdue_active_day_percent).alias("pmts_overdue_closed_day_percent"),
#         pl.map_groups(exprs=["pmts_overdue_1152A"], function=pmts_overdue_weighted_average).alias("pmts_overdue_closed_weighted_average"),

#         pl.map_groups(exprs=["pmts_dpd_303P"], function=find_ceil_sum).alias("pmts_dpd_closed"),

#         pl.map_groups(exprs=["collater_valueofguarantee_876L"], function=find_average).alias("collater_value_closed"),

#         pl.map_groups(exprs=["collater_typofvalofguarant_407M"], function=select_mask).alias("collater_typofvalofguarant_closed"),

#         pl.map_groups(exprs=["collaterals_typeofguarante_359M"], function=select_mask).alias("collaterals_typeofguarante_closed"),

#         pl.map_groups(exprs=["subjectroles_name_541M"], function=select_mask).alias("subjectroles_name_closed")
#     )
    
#     # Write the result to parquet file
#     result_df.write_parquet(f'bereau_a_2_{i+already_done}.parquet')
    
#     # Print progress
#     print(f'{i+already_done}/10')

In [35]:
# credit_bureau_a_2 = scan_data_and_parse_dates('/kaggle/working/bereau_a_2_*.parquet').lazy().collect() 

In [36]:
# credit_bureau_a_2.write_parquet('bereau_a_2.parquet')

In [37]:
# credit_bureau_a_1 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_credit_bureau_a_1_*.parquet').lazy().collect() 
# credit_bureau_a_1.write_parquet('bereau_a_1.parquet')

In [38]:
# credit_bureau_a_2 = scan_data_and_parse_dates('/kaggle/working/bereau_a_2.parquet').lazy().collect() 
# credit_bureau_a_1 = pl.read_parquet('/kaggle/working/bereau_a_1.parquet').lazy().collect() 
# app_prev = scan_data_and_parse_dates('/kaggle/working/app_prev.parquet').lazy().collect() 
# # print(len(credit_bureau_a_2))
# print(len(credit_bureau_a_1))
# print(len(app_prev))

# Bereau b_2

In [39]:
# credit_bureau_b_2 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_credit_bureau_b_2.parquet').lazy().collect() 

In [40]:
# def get_pmts_start_date(list_of_series):
#     return list_of_series[0].min()

In [41]:
# def get_pmts_end_date(list_of_series):
#     return list_of_series[0].max()

In [42]:
# def get_pmts_mean(list_of_series):
#     values = np.trim_zeros(np.nan_to_num(np.array(list_of_series[0])))
#     return np.mean(values) if len(values) > 0 else 0.
 

In [43]:
# def get_pmts_overdue_percent(list_of_series):
#     sample = np.nan_to_num(np.array(list_of_series[0]))
    
#     non_trim_length = len(sample)
#     trim_length = len(np.trim_zeros(sample))
    
#     return trim_length / non_trim_length 

In [44]:
# bereau_b_2_to_1 = credit_bureau_b_2.group_by(['case_id', 'num_group1']).agg(
#     pl.map_groups(exprs=["pmts_date_1107D"], function=get_pmts_start_date).alias("pmts_start_date"),
#     pl.map_groups(exprs=["pmts_date_1107D"], function=get_pmts_end_date).alias("pmts_end_date"), 
#     pl.map_groups(exprs=["pmts_dpdvalue_108P"], function=get_pmts_mean).alias("pmts_dpdvalue_mean"), 
#     pl.map_groups(exprs=["pmts_pmtsoverdue_635A"], function=get_pmts_mean).alias("pmts_pmtsoverdue_mean"), 
#     pl.map_groups(exprs=["pmts_pmtsoverdue_635A"], function=get_pmts_overdue_percent).alias("pmts_overdue_percent"), 
# )

In [45]:
# nm_unique = credit_bureau_b_2['case_id'].unique()
# uniqewjkds = bereau_b_2_to_1['case_id'].unique()
# print(bereau_b_2_to_1.filter(pl.col('case_id') == 11196))
# bereau_b_2_to_1.write_parquet('bereau_b_2.parquet')

# Person 2

In [46]:
# train_person_2 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_person_2.parquet').lazy().collect() 

In [ ]:
# def get_pmts_end_date(list_of_series):
#     return list_of_series[0].max()

In [47]:
# def get_mask(list_of_series):
#     count_unique = list_of_series[0].unique()
#     return list_of_series[0].mode()[0] if len(count_unique) > 1 else list_of_series[0][0]


In [48]:
# train_person_2_to_1 = train_person_2[:20].group_by(['case_id', 'num_group1']).agg(
#     pl.map_groups(exprs=["addres_district_368M"], function=get_mask).alias("addres_district"),
#     pl.map_groups(exprs=["addres_zip_823M"], function=get_pmts_end_date).alias("addres_zip"),
#     pl.map_groups(exprs=["conts_role_79M"], function=get_mask).alias("conts_role"),
#     pl.map_groups(exprs=["empls_economicalst_849M"], function=get_mask).alias("empls_economicalst"),
#     pl.map_groups(exprs=["empls_employer_name_740M"], function=get_mask).alias("empls_employer_name"),
# )


In [49]:
# train_person_2_to_1.write_parquet('person_2.parquet')

In [50]:
# unique_id = train_person_2.filter(train_person_2['relatedpersons_role_762T'] != np.nan)['case_id'].unique()
# unique_id

In [51]:
# check_ttest_by_target(unique_id)

In [52]:
# plot_categorical(scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_person_2.parquet').filter([pl.col('addres_district_368M') != 'a55475b1']).filter(pl.col('addres_district_368M') !='P125_48_164'), 'M')

# Taxes a/b/c

In [53]:
# train_tax_registry_a_1 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_tax_registry_a_1.parquet').lazy().collect().rename({
#     'amount_4527230A': 'amount',
#     'name_4527232M': 'name',
#     'recorddate_4527225D': 'record_date'
# }).select(['case_id', 'num_group1', 'name', 'amount', 'record_date'])

# train_tax_registry_b_1 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_tax_registry_b_1.parquet').lazy().collect().rename({
#     'amount_4917619A': 'amount',
#     'name_4917606M': 'name',
#     'deductiondate_4917603D': 'record_date'
# }).select(['case_id', 'num_group1', 'name', 'amount', 'record_date'])


# train_tax_registry_c_1 = scan_data_and_parse_dates('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_tax_registry_c_1.parquet').lazy().collect().rename({
#     'pmtamount_36A': 'amount',
#     'employername_160M': 'name',
#     'processingdate_168D': 'record_date'
# }).select(['case_id', 'num_group1', 'name', 'amount', 'record_date'])

In [54]:
# unique_a = train_tax_registry_a_1['case_id'].unique()
# unique_b = train_tax_registry_b_1['case_id'].unique()
# unique_c = train_tax_registry_c_1['case_id'].unique()


# commonUniqueC = unique_a.filter(unique_a.is_in(unique_c))
# commonUniqueB = unique_a.filter(unique_a.is_in(unique_b))

# reg_a = train_tax_registry_a_1.filter(~pl.col('case_id').is_in(commonUniqueC) & ~pl.col('case_id').is_in(commonUniqueB))

# print(len(unique_b) + len(unique_c) + len(reg_a['case_id'].unique()))

In [55]:
# taxes = pl.concat([reg_a, train_tax_registry_b_1, train_tax_registry_c_1])
# taxes.write_parquet('taxes.parquet')

# Static Feature Engeneering

In [57]:
static_internal_relevants = {
    "important": [
        'credamount_770A',
    ],

    "green": [
        'annuity_780A',
        'annuitynextmonth_57A',
        # 'credamount_770A',
        'currdebt_22A',
        'currdebtcredtyperange_828A',
        'disbursedcredamount_1113A',
        'downpmt_116A',
        'totalsettled_863A'
    ],

    "yellow": [
        'avginstallast24m_3658937A',
        'amtinstpaidbefduel24m_4187115A',
        'avgoutstandbalancel6m_4187114A',
        'avglnamtstart24m_4525187A',
        'inittransactionamount_650A',
        'lastapprcredamount_781A',
        'maxannuity_159A',
        'maxinstallast24m_3658928A',
        'maxlnamtstart6m_4525199A',
        'maxoutstandbalancel12m_4187113A',
        'maxpmtlast3m_4525190A',
        'totinstallast1m_4525188A',
        'maininc_215A',
        'maxdebt4_972A',
    ],

    "blue": [
        'avgpmtlast12m_4525200A',
        'lastotherinc_902A',
        'lastotherlnsexpense_631A',
        'lastrejectcredamount_222A',
        'maxannuity_4075009A',
        'price_1097A',
    ],

    "red": [
        'sumoutstandtotal_3546847A',
        'sumoutstandtotalest_4493215A',
        'totaldebt_9A', 
    ]
}


# Train Model

In [58]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)


In [83]:
class Pipeline:
    @staticmethod
    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))            

        return df
    
    @staticmethod
    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))
                df = df.with_columns(pl.col(col).dt.total_days())
                
        df = df.drop("date_decision", "MONTH")

        return df
    
    @staticmethod
    def filter_cols(df):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()

                if isnull > 0.95:
                    df = df.drop(col)

        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()

                if (freq == 1) | (freq > 200):
                    df = df.drop(col)

        
        return df

In [84]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max
    
    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

In [85]:
def feature_eng(df_base, depth_0):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
        
    for i, df in enumerate(depth_0):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
        
    df_base = df_base.pipe(Pipeline.handle_dates)
    
    return df_base

In [86]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    
    return df_data, cat_cols

In [87]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    
    if depth in [1, 2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
    
    return df

def read_files(regex_path, depth=None):
    chunks = []
    for path in glob(str(regex_path)):
        chunks.append(pl.read_parquet(path).pipe(Pipeline.set_table_dtypes))
        
    df = pl.concat(chunks, how="vertical_relaxed")
    if depth in [1, 2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
    
    return df

In [88]:
def check_feature_importances(fitted_models, features_names, PLOT_TOP_N=50, figsize=(10, 12)):
    # Initialize an empty list to store feature importances
    fitted_importances = []

    # Iterate over each fitted model
    for model in fitted_models:
        # Collect feature importances from the model
        importances = model.feature_importances_
        # Append the feature importances to the list
        fitted_importances.append(importances)


    def plot_chinese_importance(importances, features_names, figsize):
        importance_df = pd.DataFrame(data=importances, columns=features_names)
        sorted_indices = importance_df.median(axis=0).sort_values(ascending=False).index
        sorted_importance_df = importance_df.loc[:, sorted_indices]
        plot_cols = sorted_importance_df.columns[:PLOT_TOP_N]
        _, ax = plt.subplots(figsize=figsize)
        ax.grid()
        ax.set_xscale('log')
        ax.set_ylabel('Feature')
        ax.set_xlabel('Importance')
        sns.boxplot(data=sorted_importance_df[plot_cols],
                    orient='h',
                    ax=ax)
        plt.savefig('importances', bbox_inches='tight')
        plt.show()
    
    plot_chinese_importance(fitted_importances, features_names, figsize)

In [89]:
data_train_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
}

In [90]:
df_train = feature_eng(**data_train_store)
print("train data shape:\t", df_train.shape)

train data shape:	 (1526659, 224)


In [91]:
data_test_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
}

In [92]:
df_test = feature_eng(**data_test_store)
print("test data shape:\t", df_test.shape)

test data shape:	 (10, 223)


In [93]:
df_train_copy = df_train.clone()

for column in static_internal_relevants['green']:   
    df_train_copy = df_train_copy.with_columns((pl.col(column) / pl.col('credamount_770A')).alias(f'new_{column}'))

df_train_copy = df_train_copy.pipe(Pipeline.filter_cols)

In [94]:
df_test_copy = df_test.clone()

for column in static_internal_relevants['green']:   
    df_test_copy = df_test_copy.with_columns((pl.col(column) / pl.col('credamount_770A')).alias(f'new_{column}'))

df_test_copy = df_test_copy.select([col for col in df_train_copy.columns if col != "target"])

In [95]:
print("train data shape:\t", df_train_copy.shape)
print("test data shape:\t", df_test_copy.shape)

train data shape:	 (1526659, 191)
test data shape:	 (10, 190)


In [96]:
df_train_copy.columns

['case_id',
 'WEEK_NUM',
 'target',
 'month_decision',
 'weekday_decision',
 'assignmentdate_238D',
 'assignmentdate_4527235D',
 'birthdate_574D',
 'contractssum_5085716L',
 'dateofbirth_337D',
 'days120_123L',
 'days180_256L',
 'days30_165L',
 'days360_512L',
 'days90_310L',
 'description_5085714M',
 'education_1103M',
 'education_88M',
 'firstquarter_103L',
 'fourthquarter_440L',
 'maritalst_385M',
 'maritalst_893M',
 'numberofqueries_373L',
 'pmtaverage_3A',
 'pmtaverage_4527227A',
 'pmtcount_4527229L',
 'pmtcount_693L',
 'pmtscount_423L',
 'pmtssum_45A',
 'requesttype_4525192L',
 'responsedate_1012D',
 'responsedate_4527233D',
 'responsedate_4917613D',
 'secondquarter_766L',
 'thirdquarter_1082L',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuity_780A',
 'annuitynextmonth_57A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_629L',
 'applicationscnt_867L',
 'avgdbddpdlast24m_3658932P',
 'av

In [97]:
df_train, cat_cols = to_pandas(df_train_copy)
df_test, cat_cols = to_pandas(df_test_copy, cat_cols)

In [98]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 8,
    "learning_rate": 0.05,
    "n_estimators": 1000,
    "colsample_bytree": 0.8, 
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "device": "gpu",
}

In [99]:
def train_voting_model(df_train, params, show_importances = False):
    X = df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
    y = df_train["target"]
    weeks = df_train["WEEK_NUM"]

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True)


    fitted_models = []

    for idx_train, idx_valid in cv.split(X, y, groups=weeks):
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            callbacks=[lgb.log_evaluation(100), lgb.early_stopping(100)]
        )

        fitted_models.append(model)
        
    if show_importances: check_feature_importances(fitted_models, X.columns)
        
    return VotingModel(fitted_models)

In [100]:
model = train_voting_model(df_train, params)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.805032
[200]	valid_0's auc: 0.812601
[300]	valid_0's auc: 0.81505
[400]	valid_0's auc: 0.815984
[500]	valid_0's auc: 0.816124
[600]	valid_0's auc: 0.816102
[700]	valid_0's auc: 0.816098
Early stopping, best iteration is:
[664]	valid_0's auc: 0.816273
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.807526
[200]	valid_0's auc: 0.815256
[300]	valid_0's auc: 0.817467
[400]	valid_0's auc: 0.818107
[500]	valid_0's auc: 0.818594
[600]	valid_0's auc: 0.819069
[700]	valid_0's auc: 0.819221
[800]	valid_0's auc: 0.819281
[900]	valid_0's auc: 0.819034
Early stopping, best iteration is:
[818]	valid_0's auc: 0.819305
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.803917
[200]	valid_0's auc: 0.811709
[300]	valid_0's auc: 0.814158
[400]	valid_0's auc: 0.814875
[500]	valid_0's auc: 0.815139
Early stopping, best iteration is:
[482]	valid_0's auc:

In [101]:
X_test = df_test.drop(columns=["WEEK_NUM"])
X_test = X_test.set_index("case_id")

y_pred = pd.Series(model.predict_proba(X_test)[:, 1], index=X_test.index)

In [102]:
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

In [103]:
df_subm["score"] = y_pred
print("Check null: ", df_subm["score"].isnull().any())

Check null:  False


In [104]:
df_subm.head()

,score
case_id,
57543,0.01
57549,0.03
57551,0.00
57552,0.01
57569,0.08


In [105]:
df_subm.to_csv("submission.csv")

In [106]:
# !rm -rf /kaggle/working/submission.csv